# Thorpe scales - diving into the details

Here we'll examine some of the options in the Thorpe scale routine and their effect on the output.

First, we load modules and example data.

In [ ]:
import mixsea as mx
import numpy as np
import matplotlib.pyplot as plt

ctd = mx.helpers.read_ctd_testfile()

Our example data contain some NaN values, as is common with observational datasets. Below we isolate the good data by removing NaNs.

In [ ]:
notnan = np.isfinite(ctd["p"]) & np.isfinite(ctd["t"]) & np.isfinite(ctd["s"])

p = ctd["p"][notnan]
t = ctd["t"][notnan]
s = ctd["s"][notnan]
lon = ctd["lon"][0]
lat = ctd["lat"][0]

## Without the intermediate profile method

Now we run the Thorpe scale algorithm on the good test data, first without using the intermediate profile method.

In [ ]:
dnoise = 5e-4  # Noise parameter
alpha_sq = 0.9  # Square of the coefficient relating the Thorpe and Ozmidov scales.
# Background value of epsilon applied where no overturns are detected.
background_eps = np.nan
# Do not use the intermediate profile method
use_ip = False
# Critical value of the overturn ratio
Roc = 0.2

# Calculate Thorpe scales and diagnostics.
eps, n2, diag = mx.overturn.eps_overturn(
    p,
    t,
    s,
    lon,
    lat,
    dnoise=dnoise,
    alpha_sq=alpha_sq,
    Roc=Roc,
    background_eps=background_eps,
    use_ip=use_ip,
    return_diagnostics=True,
)

Lets look at some of the diagnostics, there are several available to us, all contained in a dictionary.

In [ ]:
for key in diag:
    print(key)

Some of the diagnostics are as follows:

* `eps`: the dissipation rate estimated before applying quality control flags.
* `n2`: the buoyancy frequency squared estimated before applying quality control flags.
* `Lt`: the Thorpe scale.
* `thorpe_disp`: the Thorpe displacements.
* `den`: potential density (this may appear 'steppy' due to the use of different reference pressures at for different pressure ranges).
* `dens_sorted`: sorted potential density.
* `dens_ip`: density from the intermediate profile method. (Does not appear in the list above because the intermediate profile method was not used)
* `CT_ip`: conservative temperature from the intermediate profile method. (Does not appear in the list above because intermediate profile method was not used)
* `Ro`: the overturn ratio.
* `CT_sorted`: sorted conservative temperature.
* `SA_sorted`: sorted absolute salinity.

Some of the quality control flags:

* `noise_flag`: if density difference from the top to bottom of the overturn is less than the noise parameter, this flag will be True.
* `n2_flag`: if buoyancy frequency is negative, this flag will be True.
* `ends_flag`: if an overturn contains the first or last point in the data, this flag will be True.
* `Ro_flag`: if the overturn ratio is less than the critical value, this flag will be True.

If we zoom into a segment, it is easier to understand what these diagnostics mean.

In [ ]:
# Plot only in the depth range:
z = ctd["z"][notnan]
cut = (z > 4260) & (z < 4500)
zc = z[cut]

fig, axs = plt.subplots(1, 4, sharey=True, figsize=(12, 6))
axs[0].plot(diag["dens"][cut], zc, label="density")
axs[0].plot(diag["dens_sorted"][cut], zc, label="sorted density")
axs[1].plot(diag["eps"][cut], zc, label="rejected epsilon", color="r")
axs[1].plot(eps[cut], zc, label="accepted epsilon", color="g")
axs[2].plot(diag["n2"][cut], zc, label="rejected N2", color="r")
axs[2].plot(n2[cut], zc, label="accepted N2", color="g")
axs[3].plot(diag["thorpe_disp"][cut], zc, label="Thorpe displacement")
axs[3].plot(diag["Lt"][cut], zc, label="Thorpe scale")
axs[0].invert_yaxis()
axs[0].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[1].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[2].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[3].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[0].set_ylabel("Depth [m]")
axs[0].set_xlabel("Potential density [kg m$^{-3}$]")
axs[1].set_xlabel(r"$\epsilon$ [W kg$^{-1}$]")
axs[2].set_xlabel(r"$N^2$ [rad$^2$ s$^{-2}$]")
axs[3].set_xlabel("Thorpe displacement [m]")

fig.align_labels()

Starting from the left, we have potential density and sorted potential density. Overturning patches should correspond to regions where the potential density is different from the sorted potential density. Next is the dissipation rate, $\epsilon$, plotted in green are the values that have passed quality control and in red those that were rejected by some criteria. The plot looks a bit odd because there are jumps in $\epsilon$. This is caused by small overturns next to large overturns. Ultimately, a lot of small overturns will be rejected by quality control. Third from the left is buoyancy frequency, which can be calculated in a number of different ways. We will explore this in more detail later. Next is the Thorpe displacements and Thorpe length scale, which are equal to the root mean square displacement for each overturning patch.

Now we look at some of the flags.

In [ ]:
fig, axs = plt.subplots(
    1, 5, sharey=True, figsize=(9, 6), gridspec_kw={"width_ratios": [2, 1, 1, 1, 2]}
)
axs[0].plot(diag["thorpe_disp"][cut], zc, label="Thorpe displacement")
axs[0].plot(diag["Lt"][cut], zc, label="Thorpe scale")
axs[1].plot(diag["noise_flag"][cut], zc, color="r")
axs[2].plot(diag["n2_flag"][cut], zc, color="r")
axs[3].plot(diag["ends_flag"][cut], zc, color="r")
axs[4].plot(diag["Ro"][cut], zc, label="Ro value")
axs[4].plot(diag["Ro_flag"][cut], zc, color="r", label="Ro flag")
axs[4].axvline(Roc, color="k", ls=":", label="Ro critical")
axs[0].invert_yaxis()
axs[0].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[4].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[0].set_ylabel("Depth [m]")
axs[0].set_xlabel("Thorpe displacement [m]")
axs[1].set_xlabel("Noise flag")
axs[2].set_xlabel("N2 flag")
axs[3].set_xlabel("Ends flag")
axs[4].set_xlabel("Ro flag and value")

fig.align_labels()

The noise flag is raised (True) where the density difference of the overturning region is smaller than the noise parameter. The N2 flag has not been raised anywhere, indicating there are no regions of negative buoyancy frequency. The ends flag is raised for the last overturn since it contains the end point of the data. However, it is just a warning flag and not used to quality control the output. The Ro flag is not raised because the overturn ratio is never less than the critical value.

## With the intermediate profile method

Now we run the Thorpe scale algorithm again using the intermediate profile method.

In [ ]:
dnoise = 5e-4
alpha_sq = 0.9
# Use the intermediate profile method
use_ip = True

eps, n2, diag = mx.overturn.eps_overturn(
    p,
    t,
    s,
    lon,
    lat,
    dnoise=dnoise,
    alpha_sq=alpha_sq,
    use_ip=use_ip,
    return_diagnostics=True,
)

Plot the same depth range.

In [ ]:
fig, axs = plt.subplots(1, 4, sharey=True, figsize=(12, 6))
axs[0].plot(diag["dens"][cut], zc, label="density")
axs[0].plot(diag["dens_sorted"][cut], zc, label="sorted density")
axs[0].plot(diag["dens_ip"][cut], zc, label="intermediate density")
axs[1].plot(diag["eps"][cut], zc, label="rejected epsilon", color="r")
axs[1].plot(eps[cut], zc, label="accepted epsilon", color="g")
axs[2].plot(diag["n2"][cut], zc, label="rejected N2", color="r")
axs[2].plot(n2[cut], zc, label="accepted N2", color="g")
axs[3].plot(diag["thorpe_disp"][cut], zc, label="Thorpe displacement")
axs[3].plot(diag["Lt"][cut], zc, label="Thorpe scale")
axs[0].invert_yaxis()
axs[0].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[1].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[2].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[3].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[0].set_ylabel("Depth [m]")
axs[0].set_xlabel("Potential density [kg m$^{-3}$]")
axs[1].set_xlabel(r"$\epsilon$ [W kg$^{-1}$]")
axs[2].set_xlabel(r"$N^2$ [rad$^2$ s$^{-2}$]")
axs[3].set_xlabel("Thorpe displacement [m]")

fig.align_labels()

Now we're using the intermediate profile method, the results have changed dramatically. Overturns are diagnosed from the intermediate profile of density, which looks rather different to the actual density. The overturning patches may be of different sizes to those estimated without the intermediate profile. Sorting the intermediate profile results in strange looking Thorpe displacements, because the sorting algorithm struggles when many data points have an identical value. There is an argument to be made the method should not be applied to data that has already been bin averaged (as ours has) and the method may work better on raw CTD data.

Because the intermediate profile method fundamentally changes many aspects of the Thorpe algorithm, including the size of detected overturns and the Thorpe displacements, it is not straightforward or necessarily meaningful to raise a flag for rejected overturns. Consequently, most flags are disabled when using the method and so are not shown.

## Compare methods of estimating $N^2$

There are a few different options for calculating buoyancy frequency:

In [ ]:
use_ip = False

# Choose buoyancy frequency method
Nsq_method = "teosp1"
eps_teosp1, n2_teosp1 = mx.overturn.eps_overturn(
    p, t, s, lon, lat, use_ip=use_ip, Nsq_method=Nsq_method,
)

Nsq_method = "teos"
eps_teos, n2_teos = mx.overturn.eps_overturn(
    p, t, s, lon, lat, dnoise, use_ip=use_ip, Nsq_method=Nsq_method,
)

Nsq_method = "bulk"
eps_bulk, n2_bulk = mx.overturn.eps_overturn(
    p, t, s, lon, lat, use_ip=use_ip, Nsq_method=Nsq_method,
)

Nsq_method = "endpt"
eps_endpt, n2_endpt = mx.overturn.eps_overturn(
    p, t, s, lon, lat, use_ip=use_ip, Nsq_method=Nsq_method,
)

A brief explanation of the methods:

* `teos`: This method estimates $N^2$ from the thermodynamic equation of state 2010 (TEOS), using the tempurature and salinity at the first and last point in the overturn.
* `toesp1`: This method is the same as above except it uses the temperature and salinity from the points immediately above and below overturn (p1 meaning plus 1).
* `bulk`: This method comes from <cite data-cite="Smyth2001">Smyth et al. (2001)</cite> and has the advantage of being insensitive to errors in determining the patch size. Buoyancy frequency is proportional to the root mean square density anomaly of an overturn, divided by the Thorpe scale.
* `endpt`: This simple method estimates buoyancy frequency from the potential density gradient across each overturn.

In [ ]:
# Plot only in the depth range:
z = ctd["z"][notnan]
cut = (z > 4225) & (z < 4350)
zc = z[cut]

fig, axs = plt.subplots(1, 3, sharey=True, figsize=(9, 6))
axs[0].plot(diag["dens"][cut], zc, label="density")
axs[0].plot(diag["dens_sorted"][cut], zc, label="sorted density")
axs[1].plot(eps_teosp1[cut], zc, label="teosp1")
axs[1].plot(eps_teos[cut], zc, label="teos")
axs[1].plot(eps_bulk[cut], zc, label="bulk")
axs[1].plot(eps_endpt[cut], zc, label="endpt")
axs[2].plot(n2_teosp1[cut], zc, label="teosp1")
axs[2].plot(n2_teos[cut], zc, label="teos")
axs[2].plot(n2_bulk[cut], zc, label="bulk")
axs[2].plot(n2_endpt[cut], zc, label="endpt")
axs[0].invert_yaxis()
axs[0].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[1].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[2].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[0].set_ylabel("Depth [m]")
axs[0].set_xlabel("Potential density [kg m$^{-3}$]")
axs[1].set_xlabel(r"$\epsilon$ [W kg$^{-1}$]")
axs[2].set_xlabel(r"$N^2$ [rad$^2$ s$^{-2}$]")

fig.align_labels()

The different methods can alter the dissipation rate estimate by almost an order of magnitude. The `endpt` and `teos` methods give such similar results they are indistinguishable. The `teosp1` method tends to result in higher $N^2$ estimates, because there is often a jump in stratification adjacent to an overturning patch.

## Thorpe scales from temperature only

There are two different ways of doing this:

* Provide both temperature and salinity profiles but set `overturns_from_CT = True`. In this case, overturns will be diagnosed from the conservative temperature profile. Buoyancy frequency will be estimated from the equation of state or potential density as specified by `Nsq_method`. 

* Provide as arguments to `eps_overturn` a profile of temperature and a constant salinity. Pseudo potential density will be calculated assuming a constant salinity profile. Overturns will be diagnosed from the pseudo potential density or the equation of state at constant salinity. As such, buoyancy frequency values will be incorrect in the ocean but perhaps useful for estimating $\epsilon$. It is probably a good idea to choose a plausible salinity value, such as 0 psu for a fresh water environment or 35 psu for an ocean environment.

Note that `dnoise` should describe the temperature noise if `overturns_from_CT = True` and the density noise at constant salinity if salinity is constant. 

 

In [ ]:
use_ip = False
Nsq_method = "teos"

eps_t, n2_t = mx.overturn.eps_overturn(
    p, t, 35, lon, lat, use_ip=use_ip, Nsq_method=Nsq_method,
)

eps_ts, n2_ts = mx.overturn.eps_overturn(
    p, t, s, lon, lat, dnoise=2e-4, use_ip=use_ip, Nsq_method=Nsq_method, overturns_from_CT=True,
)

eps, n2 = mx.overturn.eps_overturn(
    p, t, s, lon, lat, dnoise, use_ip=use_ip, Nsq_method=Nsq_method,
)

fig, axs = plt.subplots(1, 2, sharey=True, figsize=(6, 6))
axs[0].plot(eps_ts[cut], zc, label="overturns from CT")
axs[0].plot(eps_t[cut], zc, label="overturns from pseudo potential density")
axs[0].plot(eps[cut], zc, label="normal case")
axs[1].plot(n2_ts[cut], zc)
axs[1].plot(n2_t[cut], zc)
axs[1].plot(n2[cut], zc)
axs[0].invert_yaxis()
axs[0].legend(loc="lower left", bbox_to_anchor=(0, 1))
axs[0].set_ylabel("Depth [m]")
axs[0].set_xlabel(r"$\epsilon$ [W kg$^{-1}$]")
axs[1].set_xlabel(r"$N^2$ [rad$^2$ s$^{-2}$]")

fig.align_labels()

The smaller noise parameter in the `overturns_from_CT` case means that more overturns are found, however, $\epsilon$ tends to be a bit lower than the normal case.